In [43]:
from ortools.linear_solver import pywraplp
from ortools.graph import pywrapgraph
from ortools.constraint_solver import pywrapcp

import csv 
import pandas as pd

filename = "Downloads/result (3).csv"
result = pd.read_csv('Downloads/result (3).csv') 

address = result['origin_addresses']
node = []
for i in range(len(address)):
    node += [str(address[i])]

for i in range(len(node)):
    if node[i] != 'nan':
        print(i, ' = ', node[i])
print()

dist = result['rows__elements__|__text']
dist_1 = []
for i in range(len(dist)):
    dist_1 += [str(dist[i])]

distances = []
for i in range(0,len(dist_1)):      
    if dist_1[i][-2:] == 'ft' or dist_1[i][-2:] == 'mi':  
        distances += [dist_1[i]]
            
for i in range(len(distances)):
    if distances[i][-2:] == 'ft':
        distances[i] = 0
    else: distances[i] = float(distances[i][:-3])

unit_costs = distances
supply_values = [-50546,3281,5700,946,4928,16944,18747]
edges = [[i,j] for i in range(len(supply_values)) for j in range(len(supply_values))]
capacities = [42000]*len(edges)

#for i in range(len(edges)):
#    print(edges[i], ' = ', unit_costs[i])
#print()

solver = pywraplp.Solver('SolveIntegerProblem',pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
  
x = [];
for i in range(len(edges)):
    x += ["x" + str(edges[i])]
    
y = [];
for i in range(len(edges)):
    y += ["y" + str(edges[i])]

x_vars = []
for i in range(len(x)):
    x_vars += [solver.NumVar(0.0,42000,x[i])]
    
y_vars = []
for i in range(len(y)):
    if edges[i][0] == edges[i][1]:
        y_vars += [solver.IntVar(0,0,y[i])]
    else:
        y_vars += [solver.IntVar(0,1,y[i])]
    
z = solver.IntVar(0,solver.infinity(),"z")

objective = solver.Objective()
#objective.SetCoefficient(z,1)
for i in range(len(y_vars)):
    objective.SetCoefficient(y_vars[i],unit_costs[i])
objective.SetMinimization()

demand_constraints = []
for i in range(len(supply_values)):
    demand_constraints += [solver.Constraint(supply_values[i],supply_values[i])]
    
for i in range(len(demand_constraints)):
    for j in range(len(x)):
        if edges[j][0] == i:
            demand_constraints[i].SetCoefficient(x_vars[j],-1)
        if edges[j][1] == i:
            demand_constraints[i].SetCoefficient(x_vars[j],1)

in_constraints = []
for i in range(1,len(supply_values)):
    in_constraints += [solver.Constraint(1,1)]
    
for i in range(1,len(in_constraints)+1):
    for j in range(len(y)):
        if edges[j][1] == i:
            in_constraints[i-1].SetCoefficient(y_vars[j],1)
            
out_constraints = []
for i in range(1,len(supply_values)):
    out_constraints += [solver.Constraint(1,1)]
    
for i in range(1,len(out_constraints)+1):
    for j in range(len(y)):
        if edges[j][0] == i:
            out_constraints[i-1].SetCoefficient(y_vars[j],1)
            
usage_constraints = []
for i in range(len(x)):
    usage_constraints += [solver.Constraint(0,solver.infinity())]
    
for i in range(len(usage_constraints)):
    usage_constraints[i].SetCoefficient(x_vars[i],-1)
    usage_constraints[i].SetCoefficient(y_vars[i],42000)

truck_constraint = solver.Constraint(0,0)
for i in range(len(y)):
    if edges[i][0] == 0:
        truck_constraint.SetCoefficient(y_vars[i],1)
truck_constraint.SetCoefficient(z,-1)

result_status = solver.Solve()
assert result_status == pywraplp.Solver.OPTIMAL
assert solver.VerifySolution(1e-7, True)

#print('Number of variables =', solver.NumVariables())
#print('Number of constraints =', solver.NumConstraints())
#print('Optimal objective value = %d' % solver.Objective().Value())
#print()

variable_list = x_vars+y_vars+[z] 
#for i in range(len(variable_list)):
#    print(variable_list[i])
#print()

for variable in variable_list:
    if variable.solution_value() != 0:
        print('%s = %d' % (variable.name(), variable.solution_value()))

0  =  3251 De Forest Cir, Mira Loma, CA 91752, USA
1  =  3876 E Childs Ave, Merced, CA 95341, USA
2  =  13527 Orden Dr, Santa Fe Springs, CA 90670, USA
3  =  12006 East End Ave, Chino, CA 91710, USA
4  =  8400 W Sherman St, Tolleson, AZ 85353, USA
5  =  27565 Diaz Rd, Temecula, CA 92590, USA
6  =  15155 Northam St, La Mirada, CA 90638, USA

x[0, 3] = 28674
x[0, 5] = 21872
x[2, 1] = 3281
x[3, 6] = 27728
x[5, 4] = 4927
x[6, 2] = 8981
y[0, 3] = 1
y[0, 5] = 1
y[1, 0] = 1
y[2, 1] = 1
y[3, 6] = 1
y[4, 0] = 1
y[5, 4] = 1
y[6, 2] = 1
z = 2
